# 02 - About $\textsf{RLWE}$ and $\textsf{RGSW}$.
---

From <a href='https://eprint.iacr.org/2020/086'> Bootstrapping in FHEW-like Cryptosystems </a>

In [1]:
from note_include.Ring import *
from note_include.RLWE import *

---
#### Parameters

In [2]:
n = 16
q = 7681
std = 1

---
Simple Example of Ring

In [3]:
# Test for Ring
a = [i for i in range(n)]
b = [1 for _ in range(n)]
c = np.zeros(n)
c[1] = 1

ring_poly1    = Ring(n, q, a)
ring_poly2    = Ring(n, q, b)
ring_monomial = Ring(n, q, c)

print(ring_poly1 + ring_poly2)
print(ring_poly1 * ring_monomial)


R(n=16, q=7681, coeffs= 1 + 2x + 3x^2 + 4x^3 + 5x^4 + 6x^5 + 7x^6 + 8x^7 + 9x^8 + 10x^9 + 11x^10 + 12x^11 + 13x^12 + 14x^13 + 15x^14 + 16x^15)
R(n=16, q=7681, coeffs= 7666.0 + 0.0x + 1.0x^2 + 2.0x^3 + 3.0x^4 + 4.0x^5 + 5.0x^6 + 6.0x^7 + 7.0x^8 + 8.0x^9 + 9.0x^10 + 10.0x^11 + 11.0x^12 + 12.0x^13 + 13.0x^14 + 14.0x^15)


---
Simple example of RLWE

In [4]:
rlwe   = RLWE(n, q, std)
sk, pk = rlwe.keygen()

a                = [100 for _ in range(n)]
b                = [100 for _ in range(n)]

poly_a           = Ring(n, q, a)
poly_b           = Ring(n, q, b)

print('poly_a : ', poly_a)
print('poly_b : ', poly_b)
print('---------------------------------------------------------')

poly_a :  R(n=16, q=7681, coeffs= 100 + 100x + 100x^2 + 100x^3 + 100x^4 + 100x^5 + 100x^6 + 100x^7 + 100x^8 + 100x^9 + 100x^10 + 100x^11 + 100x^12 + 100x^13 + 100x^14 + 100x^15)
poly_b :  R(n=16, q=7681, coeffs= 100 + 100x + 100x^2 + 100x^3 + 100x^4 + 100x^5 + 100x^6 + 100x^7 + 100x^8 + 100x^9 + 100x^10 + 100x^11 + 100x^12 + 100x^13 + 100x^14 + 100x^15)
---------------------------------------------------------


----
Encryption and Decryption

In [5]:
ctxt_a           = rlwe.encrypt(poly_a, sk)
ctxt_b           = rlwe.encrypt(poly_b, sk)

ptxt_a           = rlwe.decrypt(ctxt_a, sk)
ptxt_b           = rlwe.decrypt(ctxt_b, sk)


print('ctxt_a : ', ctxt_a) # (a, b)
print('ctxt_b : ', ctxt_b) # (a, b)
print('---------------------------------------------------------')

print('ptxt_a : ',ptxt_a)
print('ptxt_b : ',ptxt_b)
print('---------------------------------------------------------')

ctxt_a :  (R(n=16, q=7681, coeffs= 1931 + 4826x + 556x^2 + 7364x^3 + 1175x^4 + 3279x^5 + 7384x^6 + 4588x^7 + 277x^8 + 3415x^9 + 2791x^10 + 4769x^11 + 961x^12 + 1039x^13 + 1069x^14 + 961x^15), R(n=16, q=7681, coeffs= 5892.0 + 3280.0x + 5886.0x^2 + 7528.0x^3 + 3279.0x^4 + 5469.0x^5 + 1907.0x^6 + 3005.0x^7 + 982.0x^8 + 1755.0x^9 + 2898.0x^10 + 5728.0x^11 + 3478.0x^12 + 4174.0x^13 + 446.0x^14 + 940.0x^15))
ctxt_b :  (R(n=16, q=7681, coeffs= 1648 + 3604x + 5255x^2 + 3392x^3 + 1222x^4 + 290x^5 + 4880x^6 + 7222x^7 + 1406x^8 + 1516x^9 + 2527x^10 + 5346x^11 + 4828x^12 + 5430x^13 + 753x^14 + 3441x^15), R(n=16, q=7681, coeffs= 5399.0 + 5753.0x + 1056.0x^2 + 55.0x^3 + 2985.0x^4 + 6100.0x^5 + 3830.0x^6 + 5444.0x^7 + 5511.0x^8 + 2327.0x^9 + 2778.0x^10 + 5190.0x^11 + 856.0x^12 + 2128.0x^13 + 5862.0x^14 + 2120.0x^15))
---------------------------------------------------------
ptxt_a :  R(n=16, q=7681, coeffs= 100.0 + 100.0x + 101.0x^2 + 98.0x^3 + 100.0x^4 + 100.0x^5 + 99.0x^6 + 100.0x^7 + 100.0x^8 + 10

---
Homomorphic addition and constant addition

In [6]:
constant         = [10 for _ in range(n)]
poly_const       = Ring(n, q, constant)

ctxt_add         = rlwe.add_ctxt_ctxt(ctxt_a, ctxt_b)
ctxt_add_const   = rlwe.add_ctxt_ptxt(ctxt_b, poly_const)

ptxt_add         = rlwe.decrypt(ctxt_add, sk)
ptxt_add_const   = rlwe.decrypt(ctxt_add_const, sk)

print('ptxt_add       : ', ptxt_add)
print('ptxt_add_const : ', ptxt_add_const)
print('---------------------------------------------------------')

ptxt_add       :  R(n=16, q=7681, coeffs= 200.0 + 202.0x + 201.0x^2 + 198.0x^3 + 200.0x^4 + 199.0x^5 + 199.0x^6 + 200.0x^7 + 200.0x^8 + 199.0x^9 + 198.0x^10 + 200.0x^11 + 200.0x^12 + 198.0x^13 + 199.0x^14 + 203.0x^15)
ptxt_add_const :  R(n=16, q=7681, coeffs= 110.0 + 112.0x + 110.0x^2 + 110.0x^3 + 110.0x^4 + 109.0x^5 + 110.0x^6 + 110.0x^7 + 110.0x^8 + 108.0x^9 + 108.0x^10 + 110.0x^11 + 109.0x^12 + 110.0x^13 + 109.0x^14 + 110.0x^15)
---------------------------------------------------------


---
Homomorphic multiplication with plaintext polynomial with small magnitude coeffcients

Note that now we are in ring $\Z_q[x]/(x^n+1)$, so multiplication between two polynomials is negacyclic convolution.

In [7]:
a                = [10 for _ in range(n)]
small_const      = [2  for _ in range(n)]
poly_a           = Ring(n, q, a)
poly_small_const = Ring(n, q, small_const)

ctxt_a           = rlwe.encrypt(poly_a, sk)
ctxt_mul         = rlwe.mult_ring_ptxt(ctxt_a, poly_small_const)
ptxt_mul         = rlwe.decrypt(ctxt_mul, sk)

print("Homomorphic result : ", ptxt_mul)
print("Pure result        : ", poly_a * poly_small_const)

Homomorphic result :  R(n=16, q=7681, coeffs= 7409.0 + 7449.0x + 7489.0x^2 + 7529.0x^3 + 7573.0x^4 + 7609.0x^5 + 7641.0x^6 + 0.0x^7 + 40.0x^8 + 80.0x^9 + 124.0x^10 + 160.0x^11 + 196.0x^12 + 236.0x^13 + 280.0x^14 + 312.0x^15)
Pure result        :  R(n=16, q=7681, coeffs= 7401.0 + 7441.0x + 7481.0x^2 + 7521.0x^3 + 7561.0x^4 + 7601.0x^5 + 7641.0x^6 + 0.0x^7 + 40.0x^8 + 80.0x^9 + 120.0x^10 + 160.0x^11 + 200.0x^12 + 240.0x^13 + 280.0x^14 + 320.0x^15)
